# Định nghĩa Model 

In [1]:
!ls

drive  sample_data


In [2]:
%cd "/content/drive/My Drive/Jobfair-Project/Vietnam-Landmarks-Detection"

/content/drive/My Drive/Jobfair-Project/Vietnam-Landmarks-Detection


In [3]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe
from PIL import Image

In [0]:
class ConvBlock(tf.keras.Model):
    def __init__(self, filters, kernel, strides, padding):
        '''
        Khởi tạo Convolution Block với các tham số đầu vào
        
        Parameters
        ----------
        filters: int
            số lượng filter
        kernel: int
            kích thước kernel
        strides: int
            stride của convolution layer
        padding: str
            Loại padding của convolution layer
        
        '''
        
        super(ConvBlock, self).__init__()
        ## TODO 2
        ### START CODE HERE
        
        # Tạo layer Conv2D
        self.cnn = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel, strides=strides, padding=padding)
        
        # Tạo layer MaxPool2D
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
        # Tạo các layer khác tùy ý nếu cần thiết
        self.skip = tf.keras.layers.Conv2D(filters=filters, kernel_size=(1,1), strides=strides, padding=padding)
        
        ### END CODE HERE
        
        
    def call(self, inputs):
        '''
        Hàm này sẽ được gọi trong quá trình forwarding của mạng
        
        Parameters
        ----------
        inputs: tensor đầu vào
        
        Returns
        -------
        tensor
            giá trị đầu ra của mạng
        '''
        
        x = self.cnn(inputs)
        # skip = self.skip(inputs)
        ## TODO 3
        ### START CODE HERE
        
        # Forward inputs qua từng layer và gán vào biến x để trả về
        
        x = tf.keras.activations.relu(x)
        # x = tf.keras.layers.Add()([x, skip])
        # x = tf.keras.activations.relu(x)
        x = self.pool(x)
        ## END CODE HERE

        return x

In [0]:
class CNN(tf.keras.Model):
    def __init__(self, num_classes):
        
        super(CNN, self).__init__()
        
        ## TODO 4
        ### START CODE HERE
        
        # Khởi tạo các convolution block
        self.block1 = ConvBlock(filters=64, kernel=(7, 7), strides=2, padding="same")
        self.block2 = ConvBlock(filters=64, kernel=(3, 3), strides=1, padding="same")
        self.block3 = ConvBlock(filters=128, kernel=(3, 3), strides=1, padding="same")
        self.block4 = ConvBlock(filters=256, kernel=(3, 3), strides=1, padding="same")
        self.block5 = ConvBlock(filters=512, kernel=(3, 3), strides=1, padding="same")
        
        # Khởi tạo layer để flatten feature map 
        self.flatten = tf.keras.layers.Flatten()
        
        ### END CODE HERE
        
        ## TODO 5
        ### START CODE HERE
        
        # Khởi tạo fully connected layer
        self.dense = tf.keras.layers.Dense(1000, kernel_regularizer=tf.keras.regularizers.l2(0.001) , activation='relu')
        self.dense1 = tf.keras.layers.Dense(num_classes)
        
        ### END CODE HERE

    def call(self, inputs):
        
        ## TODO 6
        x = inputs
        ### START CODE HERE
        
        # Forward gía trị inputs qua các tầng CNN và gán vào x
        x = self.block1(x)
        x = tf.keras.layers.Dropout(0.05)(x)
        x = self.block2(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = self.block3(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = self.block4(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = self.block5(x)

        x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
        ### END CODE HERE
        
        ## TODO 7
        
        ### START CODE HERE 
        
        # Forward giá trị x qua Fully connected layer
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dense1(x)
        
        ### END CODE HERE
        
        # Để sử dụng hàm softmax, ta phải thực thi trên CPU
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(x)

        return output

### Load Model 

In [6]:
# Load best model
model = CNN(11)
model.build((1, 224, 224, 3))
# model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.load_weights('Weight/model_weights.h5')
print("Model đã được load")

Model đã được load


In [8]:
model.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block (ConvBlock)       multiple                  9472      
_________________________________________________________________
conv_block_1 (ConvBlock)     multiple                  36928     
_________________________________________________________________
conv_block_2 (ConvBlock)     multiple                  73856     
_________________________________________________________________
conv_block_3 (ConvBlock)     multiple                  295168    
_________________________________________________________________
conv_block_4 (ConvBlock)     multiple                  1180160   
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  513000  

In [9]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

Using TensorFlow backend.


In [10]:
gdd.download_file_from_google_drive(file_id='1ycR7Aexe8xbZ8oEDsQwGc9SIiFklRpfu', dest_path='./data.zip', unzip=True)

Unzipping...Done.
